# Application


In [40]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [42]:
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

In [43]:
app.config.suppress_callback_exceptions = True

In [44]:
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

In [45]:
airline_data.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,...,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1295781,1998,2,4,2,4,1998-04-02,AS,19930,AS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1125375,2013,2,5,13,1,2013-05-13,EV,20366,EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118824,1993,3,9,25,6,1993-09-25,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634825,1994,4,11,12,6,1994-11-12,HP,19991,HP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1888125,2017,3,8,17,4,2017-08-17,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
year_list = [i for i in range(2005, 2021, 1)]

In [47]:
def compute_data_choice_1(df):
    bar_data = df.groupby(['Month','CancellationCode'])['Flights'].sum().reset_index()
    line_data = df.groupby(['Month','Reporting_Airline'])['AirTime'].mean().reset_index()
    div_data = df[df['DivAirportLandings'] != 0.0]
    map_data = df.groupby(['OriginState'])['Flights'].sum().reset_index()
    tree_data = df.groupby(['DestState', 'Reporting_Airline'])['Flights'].sum().reset_index()
    return bar_data, line_data, div_data, map_data, tree_data

In [48]:
def compute_data_choice_2(df):
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

In [49]:
app.layout = html.Div(children=[ 
    html.H1('US Domestic Airline Flights Performance', style={'textAlign':'center',
                                                             'color': '#503D36',
                                                             'font-size': 24}),
                                html.Div([
                                    html.Div([
                                        html.Div(
                                            [
                                            html.H2('Report Type:', style={'margin-right': '2em'}),
                                            ]
                                        ),
                                        dcc.Dropdown(id='input-type', options=[{'label': 'Yearly Airline Performance Report',
                                                                               'value': 'opt1'},
                                                                              {'label': 'Yearly Airline Delay Report',
                                                                              'value': 'opt2'}],
                                                    placeholder='Choose report type',
                                                    style={'width': '80%', 'padding': '3px',
                                                           'font-size': '20px', 'text-align-last': 'center'})
                                        
                                    ], style={'display':'flex'}),
                                    
                                   html.Div([
                                        html.Div(
                                            [
                                            html.H2('Choose Year:', style={'margin-right': '2em'})
                                            ]
                                        ),
                                        dcc.Dropdown(id='input-year', 
                                                     options=[{'label': i, 'value': i} for i in year_list],
                                                     placeholder="Select a year",
                                                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                            ], style={'display': 'flex'}),  
                                          ]),
                                
                                html.Div([ ], id='plot1'),
    
                                html.Div([
                                        html.Div([ ], id='plot2'),
                                        html.Div([ ], id='plot3')
                                ], style={'display': 'flex'}),
                                
                               html.Div([
                                   html.Div([ ], id='plot2'),
                                   html.Div([ ], id='plot3')
                               ], style={'display', 'flex'}),])

In [50]:
@app.callback([Output(component_id='plot1', component_property='children'),
               Output(component_id='plot2', component_property='children'),
               Output(component_id='plot3', component_property='children'),
               Output(component_id='plot4', component_property='children'),
               Output(component_id='plot5', component_property='children')],
              [Input(component_id='input-type', component_property='value'),
               Input(component_id='input-year', component_property='value')],
              [State('plot1', 'children'), State('plot2', 'children'),
               State('plot3', 'children'), State('plot4', 'children'),
               State('plot5', 'children')])

def get_graph(chart, year, children1, children2, c3, c4, c5):
    df =  airline_data[airline_data['Year']==int(year)]
    if chart == 'OPT1':
        bar_data, line_data, div_data, map_data, tree_data = compute_data_choice_1(df)
        bar_fig = px.bar(bar_data, x='Month', y='Flights', color='CancellationCode', title='Monthly Flight Cancellation')
        pie_fig = px.pie(div_data, values='Flights', names='Reporting_Airline', title='% of flights by reporting airline')
        map_fig = px.choropleth(map_data,
                                locations='OriginState', 
                                color='Flights',  
                                hover_data=['OriginState', 'Flights'], 
                                locationmode = 'USA-states',
                                color_continuous_scale='GnBu',
                                range_color=[0, map_data['Flights'].max()]) 
        map_fig.update_layout(title_text = 'Number of flights from origin state', 
                              geo_scope='usa')
        tree_fig = px.treemap(tree_data, path=['DestState', 'Reporting_Airline'],
                              values='Flights', color='Flights', color_continuous_scale='RdBu',
                              title='Flights by airline to destination States')
        return [dcc.Graph(figure=tree_fig), 
                dcc.Graph(figure=pie_fig),
                dcc.Graph(figure=map_fig),
                dcc.Graph(figure=bar_fig),
                dcc.Graph(figure=line_fig)]
    else:
        avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_data_choice_2(df)
        carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
        weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
        nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
        sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
        late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
        
        return[dcc.Graph(figure=carrier_fig),
               dcc.Graph(figure=weather_fig), 
               dcc.Graph(figure=nas_fig),
               dcc.Graph(figure=sec_fig),
               dcc.Graph(figure=late_fig)]

In [51]:
if __name__ == '__main__':
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)

127.0.0.1 - - [28/Jun/2021 14:15:03] "GET /_shutdown_92b64b54-3850-47b3-9639-cb503554ced4 HTTP/1.1" 500 -
127.0.0.1 - - [28/Jun/2021 14:15:03] "GET /_alive_92b64b54-3850-47b3-9639-cb503554ced4 HTTP/1.1" 500 -
127.0.0.1 - - [28/Jun/2021 14:15:03] "GET /_alive_92b64b54-3850-47b3-9639-cb503554ced4 HTTP/1.1" 500 -
127.0.0.1 - - [28/Jun/2021 14:15:03] "GET /_alive_92b64b54-3850-47b3-9639-cb503554ced4 HTTP/1.1" 500 -


---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
~/conda/envs/python/lib/python3.6/site-packages/flask/app.py in wsgi_app(
    self=<Flask '__main__'>,
    environ={'HTTP_ACCEPT': '*/*', 'HTTP_ACCEPT_ENCODING': 'gzip, deflate', 'HTTP_CONNECTION': 'keep-alive', 'HTTP_HOST': 'localhost:8050', 'HTTP_USER_AGENT': 'python-requests/2.25.1', 'PATH_INFO': '/_shutdown_92b64b54-3850-47b3-9639-cb503554ced4', 'QUERY_STRING': '', 'RAW_URI': '/_shutdown_92b64b54-3850-47b3-9639-cb503554ced4', 'REMOTE_ADDR': '127.0.0.1', 'REMOTE_PORT': 51000, ...},
    start_response=<function WSGIRequestHandler.run_wsgi.<locals>.start_response>
)
   2068             try:
   2069                 ctx.push()
-> 2070                 response = self.full_dispatch_request()
        response = undefined
        self.full_dispatch_request = <bound method Flask.full_dispatch_request of <Flask '__main__'>>
   2071            

127.0.0.1 - - [28/Jun/2021 14:15:03] "GET /_alive_92b64b54-3850-47b3-9639-cb503554ced4 HTTP/1.1" 500 -
127.0.0.1 - - [28/Jun/2021 14:15:03] "GET /_alive_92b64b54-3850-47b3-9639-cb503554ced4 HTTP/1.1" 500 -


---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
~/conda/envs/python/lib/python3.6/site-packages/flask/app.py in wsgi_app(
    self=<Flask '__main__'>,
    environ={'HTTP_ACCEPT': '*/*', 'HTTP_ACCEPT_ENCODING': 'gzip, deflate', 'HTTP_CONNECTION': 'keep-alive', 'HTTP_HOST': 'localhost:8050', 'HTTP_USER_AGENT': 'python-requests/2.25.1', 'PATH_INFO': '/_alive_92b64b54-3850-47b3-9639-cb503554ced4', 'QUERY_STRING': '', 'RAW_URI': '/_alive_92b64b54-3850-47b3-9639-cb503554ced4', 'REMOTE_ADDR': '127.0.0.1', 'REMOTE_PORT': 51022, ...},
    start_response=<function WSGIRequestHandler.run_wsgi.<locals>.start_response>
)
   2068             try:
   2069                 ctx.push()
-> 2070                 response = self.full_dispatch_request()
        response = undefined
        self.full_dispatch_request = <bound method Flask.full_dispatch_request of <Flask '__main__'>>
   2071             excep

127.0.0.1 - - [28/Jun/2021 14:15:04] "GET /_alive_92b64b54-3850-47b3-9639-cb503554ced4 HTTP/1.1" 500 -


...

---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
~/conda/envs/python/lib/python3.6/site-packages/flask/app.py in wsgi_app(
    self=<Flask '__main__'>,
    environ={'HTTP_ACCEPT': '*/*', 'HTTP_ACCEPT_ENCODING': 'gzip, deflate', 'HTTP_CONNECTION': 'keep-alive', 'HTTP_HOST': 'localhost:8050', 'HTTP_USER_AGENT': 'python-requests/2.25.1', 'PATH_INFO': '/_alive_92b64b54-3850-47b3-9639-cb503554ced4', 'QUERY_STRING': '', 'RAW_URI': '/_alive_92b64b54-3850-47b3-9639-cb503554ced4', 'REMOTE_ADDR': '127.0.0.1', 'REMOTE_PORT': 51086, ...},
    start_response=<function WSGIRequestHandler.run_wsgi.<locals>.start_response>
)
   2068             try:
   2069                 ctx.push()
-> 2070                 response = self.full_dispatch_request()
        response = undefined
        self.full_dispatch_request = <bound method Flask.full_dispatch_request of <Flask '__main__'>>
   2071             excep

127.0.0.1 - - [28/Jun/2021 14:15:13] "GET /_alive_92b64b54-3850-47b3-9639-cb503554ced4 HTTP/1.1" 500 -


---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
~/conda/envs/python/lib/python3.6/site-packages/flask/app.py in wsgi_app(
    self=<Flask '__main__'>,
    environ={'HTTP_ACCEPT': '*/*', 'HTTP_ACCEPT_ENCODING': 'gzip, deflate', 'HTTP_CONNECTION': 'keep-alive', 'HTTP_HOST': 'localhost:8050', 'HTTP_USER_AGENT': 'python-requests/2.25.1', 'PATH_INFO': '/_alive_92b64b54-3850-47b3-9639-cb503554ced4', 'QUERY_STRING': '', 'RAW_URI': '/_alive_92b64b54-3850-47b3-9639-cb503554ced4', 'REMOTE_ADDR': '127.0.0.1', 'REMOTE_PORT': 51090, ...},
    start_response=<function WSGIRequestHandler.run_wsgi.<locals>.start_response>
)
   2068             try:
   2069                 ctx.push()
-> 2070                 response = self.full_dispatch_request()
        response = undefined
        self.full_dispatch_request = <bound method Flask.full_dispatch_request of <Flask '__main__'>>
   2071             excep

OSError: Address 'http://localhost:8050' already in use.
    Try passing a different port to run_server.

Exception in thread Thread-20:
Traceback (most recent call last):
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/six.py", line 719, in reraise
    raise value
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/retrying.py", line 200, in call
    attem

## Summary

Congratulations for completing your dash and plotly assignment.

More information about the libraries can be found [here](https://dash.plotly.com/?utm_medium=Exinfluencer\&utm_source=Exinfluencer\&utm_content=000026UJ\&utm_term=10006555\&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDV0101ENSkillsNetwork20297740-2021-01-01)


## Author

[Saishruthi Swaminathan](https://www.linkedin.com/in/saishruthi-swaminathan/?utm_medium=Exinfluencer\&utm_source=Exinfluencer\&utm_content=000026UJ\&utm_term=10006555\&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDV0101ENSkillsNetwork20297740-2021-01-01)


## Changelog

| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 12-18-2020  | 1.0   | Nayef   | Added dataset link and upload to Git  |


## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
